In [62]:
import numpy as np
import pandas as pd
import os

from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

from alibi_detect.od import SpectralResidual

import seaborn as sns
sns.set(rc={'figure.figsize':(16,8)})

In [11]:
data_path = '../../../../data/train_data/host'
dfs = {}
for file in os.listdir(data_path):
    print('Reading ' + file)
    dfs[file[:-4]] = pd.read_csv(data_path+'/'+file) 

Reading dcos_docker.csv
Reading os_linux.csv
Reading db_oracle_11g.csv
Reading mw_redis.csv
Reading dcos_container.csv


In [20]:
def gen_train_seq(values):
    time_steps = 10

    output = []
    for i in range(len(values) - time_steps):
        output.append(values[i : (i + time_steps)])
        
    return np.stack(output)

In [41]:
for key in dfs:
    print('*'*40)
    print(key)
    df = dfs[key]
    for name in list(df.name.unique()):
        df_n = df[df.name==name]
        for cmdb_id in list(df_n.cmdb_id.unique()):
            df_nc = df_n[df_n.cmdb_id == cmdb_id]
            df_nc = df_nc.set_index('timestamp')['value']
        if np.mean(df_nc) != 0:
            df_sample = df_nc
            break

****************************************
dcos_docker
****************************************
os_linux
****************************************
db_oracle_11g
****************************************
mw_redis
****************************************
dcos_container


In [74]:
df = dfs['os_linux']
df = df[df.name == 'CPU_user_time'][df.cmdb_id=='os_012']

od = SpectralResidual(
    threshold=1.,
    window_amp=20,
    window_local=20,
    n_est_points=10,
    n_grad_points=5
)

od.infer_threshold(df['value'].values, threshold_perc=99.9)

<ipython-input-74-dd24655d4287>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[df.name == 'CPU_user_time'][df.cmdb_id=='os_012']


In [76]:
od.predict(df['value'].values,return_instance_score=True)['data']

{'instance_score': array([1.59991074, 0.81340809, 0.61092733, ..., 0.36057865, 0.46030797,
        0.84848146]),
 'feature_score': None,
 'is_outlier': array([0, 0, 0, ..., 0, 0, 0])}